<a href="https://colab.research.google.com/github/surya81/Cats-and-dogs-classification-/blob/main/cats%20and%20dogs%20classification%20using%20pre-trained%20network%20and%20data%20augmentation%20technique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cats and dogs lassification 
* Here i have used data augmentation technique through which i created 4 times of training dataset than original 
* ihave used vgg16 pre-trained neural network weight for the classification task 

In [47]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [48]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Kaggle"

In [49]:
%cd /content/gdrive/MyDrive/Kaggle

/content/gdrive/MyDrive/Kaggle


In [50]:
train_data_dir ='/content/gdrive/MyDrive/Kaggle/training_set/training_set'
test_data_dir = '/content/gdrive/MyDrive/Kaggle/test_set/test_set'

#import the library 

In [51]:
import cv2 
import time 
import random 
import numpy as np
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
from torch.utils.data import Dataset , DataLoader
from torchvision import transforms 
from torchvision.utils import make_grid
from torchvision import models
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt
from glob import glob 
device="cuda" if torch.cuda.is_available else "cpu"


In [52]:
cats= glob(train_data_dir+'/cats/*.jpg')
dogs =glob(train_data_dir+'/dogs/*.jpg')
fpaths=cats+dogs
print(fpaths)
print("no of dogs ",len(dogs))
print('no of cats',len(cats))
class_to_int = {"dogs":0,"cats":1}
int_to_class = {0:"dogs",1:"cats"}

['/content/gdrive/MyDrive/Kaggle/training_set/training_set/cats/cat.3285.jpg', '/content/gdrive/MyDrive/Kaggle/training_set/training_set/cats/cat.3541.jpg', '/content/gdrive/MyDrive/Kaggle/training_set/training_set/cats/cat.3876.jpg', '/content/gdrive/MyDrive/Kaggle/training_set/training_set/cats/cat.3788.jpg', '/content/gdrive/MyDrive/Kaggle/training_set/training_set/cats/cat.3350.jpg', '/content/gdrive/MyDrive/Kaggle/training_set/training_set/cats/cat.3443.jpg', '/content/gdrive/MyDrive/Kaggle/training_set/training_set/cats/cat.3562.jpg', '/content/gdrive/MyDrive/Kaggle/training_set/training_set/cats/cat.3667.jpg', '/content/gdrive/MyDrive/Kaggle/training_set/training_set/cats/cat.3711.jpg', '/content/gdrive/MyDrive/Kaggle/training_set/training_set/cats/cat.339.jpg', '/content/gdrive/MyDrive/Kaggle/training_set/training_set/cats/cat.3425.jpg', '/content/gdrive/MyDrive/Kaggle/training_set/training_set/cats/cat.508.jpg', '/content/gdrive/MyDrive/Kaggle/training_set/training_set/cats/ca

In [53]:
label = []
for fpath in fpaths:
  if(fpath.split("/")[-1].startswith("cats")):
    label.append("cats")
  else:
    label.append("dogs")
print(label)
print(fpaths[1])
print(label[1])



['dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs', 'dogs',

In [54]:
class cats_dogs(Dataset):
  def __init__(self,folder,transform =None,train = None):
    cats= glob(folder+'/cats/*.jpg')
    dogs =glob(folder+'/dogs/*.jpg')
    self.fpaths=cats+dogs
    self.transform = transform 
    self.train = train 
    self.label= ["cats" if(fpath.split("/")[-1].startswith("cats")) else "Dogs" for fpath in self.fpaths  ]
    self.label =[1 if item == "cats" else 0 for item in self.label]

    self.loaded_data = []
    for i in range(len(self.label)):
        img_name = self.fpaths[i]
        label = self.label[i]
        self.loaded_data.append((img_name,label))

  def __len__(self):
    return len(self.loaded_data)

  def __getitem__(self,ix):
    img_name,label = self.loaded_data[ix]  
    img = (cv2.imread(img_name))[:,:,::-1]
    img = torch.tensor(img/255)
    img = img.permute(2,0,1)
    if type(self.transform)==list:
            data=[]
            labels=[]
            for tfm in self.transform:
                img = tfm(img)
                data.append(img)
                labels.append(label)
            labels = torch.tensor(labels)
        
            return torch.stack(data, dim=0).to(device), labels.float().to(device)
    else:
           
            img = self.fpaths
      

In [55]:
t0=transforms.Compose([transforms.ToPILImage(),transforms.Resize((224,224)),
                                        transforms.ToTensor()])
t1=transforms.Compose([t0,transforms.RandomPerspective(distortion_scale=0.4, p=1.0)])
t2=transforms.Compose([transforms.Resize((224,224)),transforms.RandomCrop(96),t0])
t3=transforms.Compose([t0,transforms.RandomRotation((0,180))])
transform_tr=[t0,t1,t2,t3]
type(t0)

torchvision.transforms.transforms.Compose

In [56]:
from torch.utils.data.dataloader import default_collate

In [57]:
def my_collate_func(batch):
  batch = default_collate(batch)
  batch_size,num_aug,channels,height,width =batch[0].size()
  batch[0]= batch[0].view([batch_size*num_aug,channels,height,width])
  batch[1]=batch[1].view([batch_size*num_aug])
  return batch

In [58]:
test_dataloader = DataLoader(test_dataset,batch_size =12,collate_fn =my_collate_func,shuffle = True)

In [59]:
data,label = next(iter(test_dataloader))
print(data.size())
print(label.shape)

torch.Size([48, 3, 224, 224])
torch.Size([48])


In [60]:
def get_model():
  model = models.vgg16(pretrained =True)
  for param in model.parameters():
    model.avgpool = nn.AdaptiveAvgPool2d(output_size =(1,1))
    model.classifier = nn.Sequential(
      nn.Flatten(),
      nn.Linear(512,128),
      nn.ReLU(),
      nn.Dropout(0.4),
      nn.Linear(128,1),
      nn.Sigmoid(),
       )
    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(),lr = 1e-3)
    return model.to(device) , loss_fn,optimizer 

In [61]:
def train_batch(x,y,model,opt,loss_fn):
  model.train()
  prediction =model(x)
  y = y.unsqueeze(1)
  batch_loss = loss_fn(prediction,y)
  batch_loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  return batch_loss.item()

In [62]:
@torch.no_grad()
def accuracy(x,y,model):
  model.eval()
  prediction = model(x)
  is_correct =(prediction>0.5)==y
  return is_correct.cpu().numpy().tolist() 

In [66]:
def get_data():
  train = cats_dogs(train_data_dir,transform = transform_tr)
  train_dl = DataLoader(train,batch_size =2,shuffle = True,collate_fn = my_collate_func)
  val = cats_dogs(test_data_dir,transform = transform_tr)
  val_dl = DataLoader(val,batch_size =2,shuffle = True,collate_fn = my_collate_func)
  return train_dl,val_dl

In [64]:
@torch.no_grad()
def val_loss(x,y,model):
  prediction = model(x)
  val_loss = loss_fn(prediction,y)
  return val_loss.item()

In [67]:
train_dl,val_dl = get_data()
model,loss_fn,optimizer = get_model()
train_loss,train_accuracies = [],[]
val_loss,val_accuracies = [],[]
for epoch in range(5):
  train_epoch_losses,train_epoch_accuracies =[],[]
  val_epoch_accuracies =[]
  for ix,batch in enumerate(iter(train_dl)):
    x,y = batch
    batch_loss =train_batch(x,y,model,optimizer,loss_fn)
    train_epoch_losses.append(batch_loss)
  train_epoch_losses =np.array(train_epoch_losses).mean()

  for ix ,batch in enumerate(iter(train_dl)):
    x,y = batch 
    is_correct = accuracy(x,y,model)
    train_epoch_accuracies.append(is_correct)

  train_epoch_accuracies = np.array(train_epoch_accuracies).mean()

  for ix , batch in enumerate(iter(val_dl)):
    x,y = batch 
    val_is_correct = accuracy(x,y,model)
    val_epoch_accuracies.append(val_is_correct)

  val_epoch_accuracies = np.array(val_epoch_accuracies).mean()

  train_loss.append(train_epoch_losses)
  train_accuracies.append(train_epoch_accuracies)
  val_accuracies.append(val_epoch_accuracies)

RuntimeError: ignored